#### Middleware

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

#### Summarization Middleware

In [11]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import InMemorySaver ## To add short memory which enables agents to track multi-turn conversations


agent = create_agent(
    model="groq:qwen/qwen3-32b",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="groq:qwen/qwen3-32b",
            trigger=("messages", 6),
            keep=("messages", 2)
        )
    ]
)


In [12]:
config = {"configurable": {"thread_id": "test-1"}}

In [13]:
questions = [
    "what is AI?",
    "what is generative AI?",
    "what is machine learning?",
    "what is deep learning?"
]

for q in questions:
    response = agent.invoke({"messages":[HumanMessage(content=q)]}, config=config)
    print(response['messages'])
    print(f"length of messages: {len(response['messages'])}")

[HumanMessage(content='what is AI?', additional_kwargs={}, response_metadata={}, id='f52f78de-f4b2-412e-820c-9b4cfb63c27b'), AIMessage(content='<think>\nOkay, the user is asking what AI is. Let me start by defining AI clearly. Artificial Intelligence, or AI, refers to systems or machines that can perform tasks requiring human-like intelligence. But I should break that down more.\n\nFirst, I need to mention the main areas AI deals with, like learning, reasoning, problem-solving, perception, and language understanding. Maybe give examples of each to make it concrete. For instance, machine learning is a subset where systems learn from data, like recommendation systems on Netflix or Spotify.\n\nI should differentiate between different types of AI. There\'s narrow AI, which is what\'s in use today for specific tasks, and general AI, which is the hypothetical future version that can handle any intellectual task a human can. It\'s important to note that we don\'t have general AI yet.\n\nAppli

#### Summarization based on token size

In [17]:
from langchain_core.tools import tool

@tool
def search_hotels(city:str) -> str:
    """Search hotels - returns long response to use more tokens."""
    return f"""Hotels in {city}:
    1. Sheraton - 5 star, $500/night
    2. City inn - 4 star, $200/night
    3. Budget stay - 3 star, $50/night"""

agent = create_agent(
    model="groq:qwen/qwen3-32b",
    tools=[search_hotels],
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="groq:qwen/qwen3-32b",
            trigger=("tokens", 550),
            keep=("tokens", 200)
        )
    ]
)

config = {"configurable": {"thread_id": "test-2"}}

## Function to count the tokens
def count_tokens(messages):
    total = sum(len(str(m.content)) for m in messages)
    return total // 4

In [18]:
# Run test
cities = ["Paris", "London", "Tokyo", "New York", "Dubai", "Singapore"]

for city in cities:
    response = agent.invoke(
        {"messages": [HumanMessage(content=f"Find hotels in {city}")]},
        config=config
    )
    
    tokens = count_tokens(response["messages"])
    print(f"{city}: ~{tokens} tokens, {len(response['messages'])} messages")
    print(f"{(response['messages'])}")

Paris: ~105 tokens, 4 messages
[HumanMessage(content='Find hotels in Paris', additional_kwargs={}, response_metadata={}, id='c43dcfde-1961-4286-96ef-647590ba782f'), AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user wants to find hotels in Paris. Let me check the available tools. There\'s a function called search_hotels that takes a city parameter. The description says it returns a long response to use more tokens, so maybe I should call that. The required parameter is the city, which the user provided as Paris. I need to make sure the arguments are correctly formatted. Let me structure the JSON with "name" as "search_hotels" and "arguments" including "city": "Paris". That should do it.\n', 'tool_calls': [{'id': 'jyg6ffqdj', 'function': {'arguments': '{"city":"Paris"}', 'name': 'search_hotels'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 155, 'total_tokens': 288, 'completion_time': 0.211446731, 'comple

KeyboardInterrupt: 

#### Based on fraction

In [20]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver

@tool
def search_hotels(city: str) -> str:
    """Search hotels."""
    return f"Hotels in {city}: Grand Hotel $350, City Inn $180, Budget Stay $75"

# LOW fraction for testing!
agent = create_agent(
    model="groq:qwen/qwen3-32b",
    tools=[search_hotels],
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="groq:qwen/qwen3-32b",
            trigger=("fraction", 0.005),  # 0.5% = ~640 tokens ## fraction is based on the context of the LLM model
            keep=("fraction", 0.002),     # 0.2% = ~256 tokens
        ),
    ],
)

config = {"configurable": {"thread_id": "test-1"}}

# Token counter
def count_tokens(messages):
    return sum(len(str(m.content)) for m in messages) // 4

# Test
cities = ["Paris", "London", "Tokyo", "New York", "Dubai", "Singapore"]

for city in cities:
    response = agent.invoke(
        {"messages": [HumanMessage(content=f"Hotels in {city}")]},
        config=config
    )
    tokens = count_tokens(response["messages"])
    fraction = tokens / 128000  # groq:qwen/qwen3-32b context
    print(f"{city}: ~{tokens} tokens ({fraction:.4%}), {len(response['messages'])} msgs")
    print(response['messages'])

Paris: ~65 tokens (0.0508%), 4 msgs
[HumanMessage(content='Hotels in Paris', additional_kwargs={}, response_metadata={}, id='90e7509e-b86d-4a61-a3ce-556e4c425f1e'), AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user is asking for hotels in Paris. Let me check the available tools. There's a function called search_hotels that takes a city parameter. Since the user specified Paris, I need to call this function with the city set to Paris. I'll make sure the parameters are correctly formatted as JSON and that I include the required city field. Alright, that should do it.\n", 'tool_calls': [{'id': 'k5etn9pbk', 'function': {'arguments': '{"city":"Paris"}', 'name': 'search_hotels'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 147, 'total_tokens': 250, 'completion_time': 0.162973817, 'completion_tokens_details': {'reasoning_tokens': 78}, 'prompt_time': 0.005846763, 'prompt_tokens_details': None, 'queue_time': 0.

In [21]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

def read_email_tool(email_id: str) -> str:
    """Mock function to read an email by its ID."""
    return f"Email content for ID: {email_id}"

def send_email_tool(recipient: str, subject: str, body: str) -> str:
    """Mock function to send an email."""
    return f"Email sent to {recipient} with subject '{subject}'"

In [22]:
agent=create_agent(
    model="groq:qwen/qwen3-32b",
    tools=[read_email_tool,send_email_tool],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool":{
                    "allowed_decisions":["approve","edit","reject"]
                },
                "read_email_tool":False,

            }
        )
    ]
)

In [23]:
config = {"configurable": {"thread_id": "test-approve"}}
# Step 1: Request
result = agent.invoke(
    {"messages": [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?'")]},
    config=config
)

result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='5f986ecc-9881-4ee8-9ac3-c558e7886592'),
  AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user wants to send an email to john@test.com with the subject 'Hello' and body 'How are you?'. Let me check the available tools. There's a send_email_tool that requires recipient, subject, and body. All three are required. The parameters are there: recipient is the email address, subject is 'Hello', and body is 'How are you?'. I need to structure the function call correctly. Make sure the JSON has the right keys and values. No need for any other tools since the user is sending a new email, not reading one. So, the tool call should be send_email_tool with those arguments.\n", 'tool_calls': [{'id': 'hp3wycdsv', 'function': {'arguments': '{"body":"How are you?","recipient":"john@test.com","subject":"Hello"}', 'name': 's

In [24]:
from langgraph.types import Command
# Step 2: Approve
if "__interrupt__" in result:
    print("⏸️ Paused! Approving...")
    
    result = agent.invoke(
        Command(
            resume={
                "decisions": [
                    {"type": "approve"}
                ]
            }
        ),
        config=config
    )
    
    print(f"✅ Result: {result['messages'][-1].content}")

⏸️ Paused! Approving...
✅ Result: The email has been successfully sent to john@test.com with the subject "Hello".


In [25]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='5f986ecc-9881-4ee8-9ac3-c558e7886592'),
  AIMessage(content='', additional_kwargs={'reasoning_content': "Okay, the user wants to send an email to john@test.com with the subject 'Hello' and body 'How are you?'. Let me check the available tools. There's a send_email_tool that requires recipient, subject, and body. All three are required. The parameters are there: recipient is the email address, subject is 'Hello', and body is 'How are you?'. I need to structure the function call correctly. Make sure the JSON has the right keys and values. No need for any other tools since the user is sending a new email, not reading one. So, the tool call should be send_email_tool with those arguments.\n", 'tool_calls': [{'id': 'hp3wycdsv', 'function': {'arguments': '{"body":"How are you?","recipient":"john@test.com","subject":"Hello"}', 'name': 's